In [1]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

from vit_pytorch.efficient import ViT


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
!nvidia-smi

In [2]:
!wget https://raw.githubusercontent.com/lucidrains/vit-pytorch/main/examples/cats_and_dogs.ipynb

--2023-04-05 16:48:34--  https://raw.githubusercontent.com/lucidrains/vit-pytorch/main/examples/cats_and_dogs.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1111028 (1.1M) [text/plain]
Saving to: ‘cats_and_dogs.ipynb’

100%[======================================>] 1,111,028   --.-K/s   in 0.05s   

2023-04-05 16:48:34 (23.4 MB/s) - ‘cats_and_dogs.ipynb’ saved [1111028/1111028]

